# Callbacks

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [5]:
import os
from langchain_core.runnables import RunnableConfig
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from pydantic import SecretStr


llm = ChatOpenAI(
    model=os.getenv("MODEL", ""),
    base_url=os.getenv("API_URL", ""),
    api_key=SecretStr(os.getenv("API_KEY", "")),
    temperature=0
)

handler = StdOutCallbackHandler()

prompt_template = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")

# The callbacks can be passed to the chain
chain = LLMChain(llm=llm, prompt=prompt_template, callbacks=[handler])

config = RunnableConfig(
  callbacks=[handler]
)

# chain.invoke(input={"input": "rabbit"}, config=config)

# Called without the RunnableConfig handler
chain.invoke(input={"input": "rabbit"},)



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a joke about rabbit

> Finished chain.


{'input': 'rabbit',
 'text': 'Why did the rabbit bring a ladder to the party?\n\nBecause it heard the drinks were on the house! 🐇🍹'}

In [6]:
from langchain.callbacks.base import BaseCallbackHandler


class MyCustomHandler(BaseCallbackHandler):
    # Can be useful if we want to log how many
    # tokens has been used
    def on_llm_end(self, response, **kwargs) -> None:
        print(f"REPONSE: ", response)

handler = MyCustomHandler()

config = RunnableConfig(
  callbacks=[handler]
)

In [7]:
chain.invoke(input={"input": "rabbit"}, config=config)



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a joke about rabbit
REPONSE:  generations=[[ChatGeneration(text='Why did the rabbit bring a ladder to the party?\n\nBecause it heard the drinks were on the house! 🐇🍹', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Why did the rabbit bring a ladder to the party?\n\nBecause it heard the drinks were on the house! 🐇🍹', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 73, 'total_tokens': 112, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'openai/gpt-oss-20b', 'id': 'chatcmpl-7f8m6ig57lem5sna9adsmp', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--abf4c46c-8dab-488b-94a7-b52eea5a6091-0', usage_metadata={'input_tokens': 73, 'output_tokens': 39, 'total_tokens': 112, 'input_token_details': {}, 'output_token_details': {

{'input': 'rabbit',
 'text': 'Why did the rabbit bring a ladder to the party?\n\nBecause it heard the drinks were on the house! 🐇🍹'}

Langchain has got a convenice context manager function to make it easy to track costs, token usage etc.

In [9]:
from langchain_community.callbacks.manager import get_openai_callback

with get_openai_callback() as cb:
    chain.invoke(input={"input": "rabbit"})

print(cb.total_cost)



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a joke about rabbit

> Finished chain.
0.0
